# Visualize data on an interactive map

This notebook uses `lonboard` for interactive visualisation of data.

In [1]:
import geopandas as gpd
import numpy as np
import lonboard
from core.utils import used_keys
from lonboard.colormap import apply_continuous_cmap
import matplotlib as mpl
from mapclassify import classify
from sidecar import Sidecar

Define data path

In [2]:
chars_dir = "/data/uscuni-ulce/processed_data/chars/"

Define region

In [3]:
region = 69333

## Buildings
Load building data and ensure the geometries are all valid Polygons.

In [5]:
buildings = gpd.read_parquet(f"{chars_dir}buildings_chars_{region}.parquet").to_crs(4326).reset_index()

buildings.geometry = buildings.make_valid()

buildings = buildings[buildings.geom_type.str.contains("Polygon")]

Create a lonboard layer

In [6]:
%%time
layer = lonboard.PolygonLayer.from_geopandas(buildings, opacity=.3)

CPU times: user 1.69 s, sys: 259 ms, total: 1.95 s
Wall time: 1.91 s


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [7]:
sc = Sidecar(title='buildings')

Create a Map object

In [8]:
m = lonboard.Map(layer)

Display map within the sidecar plugin

In [9]:
with sc:
    display(m)

List avaialable columns

In [10]:
buildings.columns

Index(['index', 'gml_id', 'description', 'beginLifespanVersion',
       'conditionOfConstruction', 'localId', 'namespace', 'versionId',
       'referenceGeometry', 'horizontalGeometryEstimatedAccuracy', 'x', 'y',
       'id', 'iid', 'geometry', 'ssbCCo', 'ssbCor', 'ssbSqu', 'ssbCCM',
       'ssbCCD', 'sdbAre', 'sdbPer', 'sdbCoA', 'ssbERI', 'ssbElo', 'stbOri',
       'mtbSWR', 'libNCo', 'ldbPWL', 'mibCou', 'mibLen', 'mibAre', 'mibElo',
       'mibERI', 'mibCCo', 'mibLAL', 'mibFR', 'mibSCo', 'ltcBuA', 'mtbAli',
       'mtbNDi', 'ltbIBD', 'stbCeA', 'nID', 'stbSAl', 'nodeID'],
      dtype='object')

Specify a column and pass its values into a choropleth representation within the map. 

In [11]:
# buildings.explore()

In [16]:
column = 'mtbSWR'
used_keys[column]

'shared walls ratio of buildings'

In [17]:
classifier = classify(buildings[column], 'quantiles', k=50)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_fill_color = apply_continuous_cmap(vals, mpl.colormaps['viridis'])

/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/mapclassify/classifiers.py:1653: UserWarning: Not enough unique values in array to form 50 classes. Setting k to 4.
  self.bins = quantile(y, k=k)


In [13]:
buildings[column].describe().iloc[1:]

mean    0.698909
std     0.189620
min     0.038621
25%     0.569519
50%     0.718208
75%     0.852572
max     0.999996
Name: mibElo, dtype: float64

## Tessellation

Load tessellation data and ensure the geometries are all valid Polygons.

In [4]:
tess = gpd.read_parquet(f"{chars_dir}tessellations_chars_{region}.parquet").to_crs(4326)

tess.geometry = tess.make_valid()

tess = tess[tess.geom_type.str.contains("Polygon")]

Create a lonboard layer

In [5]:
%%time
layer = lonboard.SolidPolygonLayer.from_geopandas(tess, opacity=.2)

CPU times: user 4.89 s, sys: 341 ms, total: 5.23 s
Wall time: 5.24 s


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [14]:
sc = Sidecar(title='tess')

Create a Map object

In [7]:
m = lonboard.Map(layer)

Display map within the sidecar plugin

In [15]:
with sc:
    display(m)

List avaialable columns

In [9]:
tess.columns

Index(['enclosure_index', 'geometry', 'stcOri', 'sdcLAL', 'sdcAre', 'sscCCo',
       'sscERI', 'mtcWNe', 'mdcAre', 'ltcWRB', 'sicCAR', 'barea', 'micBAD',
       'licBAD', 'stcSAl', 'nID', 'nodeID'],
      dtype='object')

In [12]:
column = 'barea'
used_keys[column]

KeyError: 'barea'

Specify a column and pass its values into a choropleth representation within the map. 

In [13]:

classifier = classify(tess[column], 'quantiles', k=40)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_fill_color = apply_continuous_cmap(vals, mpl.colormaps['viridis'])

## Enclosures

Load data and ensure the geometries are all valid Polygons.

In [35]:
enc = gpd.read_parquet(f"{chars_dir}enclosures_chars_{region}.parquet").to_crs(4326)

enc.geometry = enc.make_valid()

enc = enc[enc.geom_type.str.contains("Polygon")]

Create a lonboard layer

In [36]:
%%time
layer = lonboard.SolidPolygonLayer.from_geopandas(enc, opacity=.3)

CPU times: user 168 ms, sys: 31.9 ms, total: 200 ms
Wall time: 200 ms


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [37]:
sc = Sidecar(title='enclosures')

Create a Map object

In [38]:
m = lonboard.Map(layer)

Display map within the sidecar plugin

In [39]:
with sc:
    display(m)

List avaialable columns

In [40]:
enc.columns

Index(['eID', 'geometry', 'ldkAre', 'ldkPer', 'lskCCo', 'lskERI', 'lskCWA',
       'ltkOri', 'ltkWNB', 'likWCe', 'likWBB'],
      dtype='object')

In [41]:
column = 'ldkPer'
used_keys[column]

'perimeter of enclosure'

Specify a column and pass its values into a choropleth representation within the map. 

In [42]:

classifier = classify(enc[column], 'quantiles', k=20)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_fill_color = apply_continuous_cmap(vals, mpl.colormaps['viridis'])

## Streets

Load data and ensure the geometries are all valid Polygons.

In [18]:
streets = gpd.read_parquet(f"{chars_dir}streets_chars_{region}.parquet")

streets.geometry = streets.make_valid()

Create a lonboard layer

In [19]:
%%time
layer = lonboard.PathLayer.from_geopandas(streets.to_crs(4326), width_min_pixels=1)

CPU times: user 118 ms, sys: 9.98 ms, total: 128 ms
Wall time: 127 ms


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [20]:
sc = Sidecar(title='streets')

Create a Map object

In [21]:
m = lonboard.Map(layer)

Display map within the sidecar plugin

In [22]:
with sc:
    display(m)

List avaialable columns

In [23]:
assert np.allclose(streets['sdsLen'] , streets.geometry.length)

if 'mm_len' in streets.columns:
    assert np.allclose(streets['mm_len'] , streets.geometry.length)

In [24]:
streets.columns

Index(['geometry', '_status', 'mm_len', 'cdsbool', 'node_start', 'node_end',
       'sdsLen', 'sssLin', 'ldsMSL', 'sdsAre', 'ldsRea', 'ldsAre', 'sisBpM',
       'misBAD', 'sdsSPW', 'sdsSPO', 'sdsSWD', 'nID'],
      dtype='object')

In [25]:
streets[['sdsLen', 'sssLin', 'ldsMSL', 'sdsAre', 'ldsRea', 'ldsAre',
       'sisBpM', 'sdsSPW', 'sdsSPO', 'sdsSWD']].describe()

,sdsLen,sssLin,ldsMSL,sdsAre,ldsRea,ldsAre,sisBpM,sdsSPW,sdsSPO,sdsSWD
count,24155.000000,24155.000000,24155.000000,1.968500e+04,24155.000000,2.245400e+04,19046.000000,24155.000000,24155.000000,17632.000000
mean,386.031895,0.957977,399.593607,5.395904e+04,264.258456,9.260281e+05,0.063553,36.597900,0.821258,3.913480
std,629.662917,0.099379,349.484916,9.218028e+04,175.471539,9.230841e+05,0.052849,11.300294,0.183244,1.956093
min,1.779955,0.000000,22.997029,2.226503e-01,0.000000,3.801642e+03,0.000167,0.000000,0.050000,0.000000
25%,98.983143,0.970070,169.476379,8.441501e+03,147.000000,3.116659e+05,0.025527,26.756167,0.687500,2.555375
50%,193.610800,0.997328,250.932766,2.240300e+04,242.000000,6.098273e+05,0.052691,37.182577,0.869919,4.062591
75%,377.847408,0.999998,504.787106,5.820301e+04,354.000000,1.195124e+06,0.089664,50.000000,1.000000,5.386908
max,16951.254801,1.000000,6126.390497,1.758182e+06,1377.000000,8.037908e+06,1.846862,50.000000,1.000000,11.008117


Specify a column and pass its values into a choropleth representation within the map. 

In [29]:
column = 'sdsSPW'
used_keys[column]

'width of street profile'

In [30]:
streets[column] = streets[column].fillna(0)

In [31]:
classifier = classify(streets[column].astype(int), 'equalinterval', k=50)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_color = apply_continuous_cmap(vals, mpl.colormaps['viridis'])

## Nodes

Load data and ensure the geometries are all valid Polygons.

In [32]:
nodes = gpd.read_parquet(f"{chars_dir}nodes_chars_{region}.parquet").to_crs(4326)

Create a lonboard layer

In [33]:
%%time
layer = lonboard.ScatterplotLayer.from_geopandas(nodes, radius_min_pixels=2)

CPU times: user 33.8 ms, sys: 2.97 ms, total: 36.8 ms
Wall time: 36.5 ms


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [34]:
sc = Sidecar(title='nodes')

Create a Map object

In [35]:
m = lonboard.Map(layer, basemap_style=lonboard.basemap.CartoBasemap.Positron)

Display map within the sidecar plugin

In [36]:
with sc:
    display(m)

List avaialable columns

In [37]:
nodes.columns

Index(['x', 'y', 'mtdDeg', 'lcdMes', 'linP3W', 'linP4W', 'linPDE', 'lcnClo',
       'lddNDe', 'linWID', 'ldsCDL', 'xcnSCl', 'mtdMDi', 'nodeID', 'geometry',
       'midBAD', 'sddAre', 'midRea', 'midAre'],
      dtype='object')

Specify a column and pass its values into a choropleth representation within the map. 

In [38]:
nodes[['mtdDeg', 'lcdMes', 'linP3W', 'linP4W', 'linPDE', 'lcnClo',
       'lddNDe', 'linWID', 'ldsCDL', 'xcnSCl', 'mtdMDi', 'nodeID', 'geometry',
       'sddAre', 'midRea', 'midAre']].describe()

,mtdDeg,lcdMes,linP3W,linP4W,linPDE,lcnClo,lddNDe,linWID,ldsCDL,xcnSCl,mtdMDi,nodeID,sddAre,midRea,midAre
count,19496.000000,19496.000000,19496.000000,19496.000000,19496.000000,1.949600e+04,19496.000000,19496.000000,19496.000000,19496.000000,19496.000000,19496.00000,1.711600e+04,19496.000000,1.765900e+04
mean,2.477944,0.085284,0.682505,0.104283,0.212255,1.887256e-06,0.003526,0.005831,882.309517,0.021187,386.081412,9747.50000,6.205794e+04,44.577247,1.552534e+05
std,1.014198,0.054032,0.118264,0.083171,0.110212,1.405456e-06,0.002425,0.003555,1289.245276,0.047371,518.177763,5628.15476,9.149249e+04,41.710329,1.992186e+05
min,1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000286,0.000000,0.000000,0.000000,3.099139,0.00000,6.428662e+00,0.000000,2.219230e+02
25%,1.000000,0.047059,0.622951,0.045455,0.142857,7.424187e-07,0.001678,0.002700,229.506759,0.000000,133.671630,4873.75000,1.330871e+04,14.000000,3.685127e+04
50%,3.000000,0.081081,0.695652,0.090909,0.200000,1.530668e-06,0.003297,0.005482,500.392489,0.000000,210.929432,9747.50000,3.054813e+04,35.000000,8.467917e+04
75%,3.000000,0.117647,0.760000,0.146341,0.266667,2.713025e-06,0.004740,0.008229,993.203161,0.000000,395.362400,14621.25000,7.022078e+04,63.000000,1.880395e+05
max,5.000000,0.352941,1.000000,0.550000,1.000000,9.021213e-06,0.062490,0.048315,26019.573144,0.500000,16951.254801,19495.00000,1.280031e+06,457.000000,2.196893e+06


In [39]:
column = 'mtdMDi'
used_keys[column]

'mean distance to neighbouring nodes of street network'

In [40]:

classifier = classify(nodes[column], 'quantiles', k=20)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_fill_color = apply_continuous_cmap(vals, mpl.colormaps['viridis'])

## Visualize merged data

In [6]:
import pandas as pd

primary = pd.read_parquet(chars_dir + f'primary_chars_{region}.parquet')

In [7]:
tess = gpd.read_parquet(f"{chars_dir}tessellations_chars_{region}.parquet").to_crs(4326)
tess.geometry = tess.make_valid()
tess = tess[tess.geom_type.str.contains("Polygon")]
tess = tess[['enclosure_index','geometry']]

In [8]:
tess = tess.join(primary)

Create a lonboard layer

In [9]:
%%time
layer = lonboard.SolidPolygonLayer.from_geopandas(tess, opacity=.2)

CPU times: user 5.93 s, sys: 436 ms, total: 6.36 s
Wall time: 6.36 s


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [10]:
sc = Sidecar(title='tess')

Create a Map object

In [11]:
m = lonboard.Map(layer)

Display map within the sidecar plugin

In [12]:
with sc:
    display(m)

List avaialable columns

In [13]:
tess.columns

Index(['enclosure_index', 'geometry', 'sdbAre', 'sdbPer', 'sdbCoA', 'ssbCCo',
       'ssbCor', 'ssbSqu', 'ssbERI', 'ssbElo', 'ssbCCM', 'ssbCCD', 'stbOri',
       'mtbSWR', 'libNCo', 'ldbPWL', 'ltcBuA', 'mtbAli', 'mtbNDi', 'ltbIBD',
       'stbCeA', 'stbSAl', 'sdsLen', 'sssLin', 'ldsMSL', 'ldsRea', 'ldsAre',
       'sisBpM', 'sdsSPW', 'sdsSPO', 'sdsSWD', 'mtdDeg', 'lcdMes', 'linP3W',
       'linP4W', 'linPDE', 'lcnClo', 'lddNDe', 'linWID', 'ldsCDL', 'xcnSCl',
       'mtdMDi', 'sddAre', 'midRea', 'midAre', 'stcOri', 'sdcLAL', 'sdcAre',
       'sscCCo', 'sscERI', 'mtcWNe', 'mdcAre', 'ltcWRB', 'sicCAR', 'stcSAl',
       'ldkAre', 'ldkPer', 'lskCCo', 'lskERI', 'lskCWA', 'ltkOri', 'ltkWNB',
       'likWBB', 'sdsAre', 'likWCe', 'mibCou', 'mibAre', 'mibLen', 'mibElo',
       'mibERI', 'mibCCo', 'mibLAL', 'mibFR', 'mibSCo', 'micBAD', 'licBAD',
       'misBAD', 'midBAD'],
      dtype='object')

In [36]:
graph_dir = '/data/uscuni-ulce/processed_data/neigh_graphs/'
from libpysal.graph import read_parquet, Graph
graph = read_parquet(graph_dir + f'tessellation_graph_{region}.parquet')
graph3 = graph.higher_order(k=3, lower_order=True, diagonal=True)

In [45]:
tess['mibElo3'] = graph3.describe(tess['mibElo'])['median']

In [50]:
tess['mibERI3'] = graph3.describe(tess['mibERI'])['median']

In [51]:
column = 'mibERI3'
used_keys[column]

KeyError: 'mibERI3'

In [52]:
tess[column].describe()

count    441409.000000
mean          0.935969
std           0.113091
min           0.112392
25%           0.931576
50%           0.963585
75%           0.992682
max           1.126041
Name: mibERI3, dtype: float64

Specify a column and pass its values into a choropleth representation within the map. 

In [53]:

classifier = classify(tess[column], 'quantiles', k=100)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_fill_color = apply_continuous_cmap(vals, mpl.colormaps['viridis'])

In [24]:
## ldsAre and sdsAre, ldsRea seperate atleast somewhat apartments and industrial, 
# maybe add sdsRea or 
#the same stuff except for connected buildings
# or reached unique streets
# mtdMDi has potential, but we have to change it to something like distance to nearest 8 nodes or something
# square clustering does nothing, looks weird on the graph, probably something with grids...
# ldsCDL okish
# linWID oksih
# lddNDe okish
# 'local closeness of street network' suprisingly ok, but not enough variability to do anything
#meshedness bad
# degree useless

# ldbPWL - miht make thigs worse
# street profile is not great for this
# mtbNDi meh
# ltbIBD meh


# midrea and mid area are meh, stds might be more usefull
# cell area, lal not usefull
# mtcWNe not useful
# mdcAre - meh
# ltcWRB - meh
